In [1]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np
import random

In [2]:
LFP = Structure.from_file("/data/kevinxhan/dist_chgnet/LiFePO4.cif")

In [3]:
s = LFP.make_supercell((10, 10, 10), in_place=False)
print("# of atoms in system:", len(s))
print(s.composition)
print(s.lattice)
n_Li = 4000
site_indices_to_remove = random.sample(range(0,n_Li), int(n_Li/2))
s.remove_sites(site_indices_to_remove)
print(s.composition)
print("# of atoms in system:", len(s))
supercell = s.make_supercell([2, 1, 1], in_place=False)
print("# of atoms in system:", len(supercell))
print(supercell.composition)
print(supercell.lattice)
LFP_atoms = AseAtomsAdaptor().get_atoms(supercell)
print(len(LFP_atoms))

# of atoms in system: 28000
Li+4000 Fe2+4000 P5+4000 O2-16000
46.549172 0.000000 0.000000
0.000000 59.707551 0.000000
0.000000 0.000000 102.361961
Li+2000 Fe2+4000 P5+4000 O2-16000
# of atoms in system: 26000
# of atoms in system: 52000
Li+4000 Fe2+8000 P5+8000 O2-32000
93.098344 0.000000 0.000000
0.000000 59.707551 0.000000
0.000000 0.000000 102.361961
52000


In [4]:
error_struct = Structure.from_file("/data/shared/Bowen_data/Kevin/bug_structure_4.30_POSCAR")

In [ ]:
# create a larger system to preallocate memory with
s = LFP.make_supercell((12, 12, 12), in_place=False)
n_Li = 4000
site_indices_to_remove = random.sample(range(0,n_Li), int(n_Li/2))
s.remove_sites(site_indices_to_remove)
supercell_prealloc = s.make_supercell([2, 1, 1], in_place=False)

supercell_prealloc = supercell_prealloc.scale_lattice(supercell_prealloc.volume * 0.70)

LFP_atoms_prealloc = AseAtomsAdaptor().get_atoms(supercell_prealloc)
print(len(LFP_atoms_prealloc))

In [ ]:
len(supercell)/supercell.volume, len(supercell_prealloc) / supercell_prealloc.volume

In [6]:
import matgl
from DistMLIP.implementations.matgl import Potential_Dist, PESCalculator_Dist
from DistMLIP.implementations.matgl import CHGNet_Dist, TensorNet_Dist
from DistMLIP.implementations.matgl import MolecularDynamics, Relaxer


model = matgl.load_model('CHGNet-MPtrj-2023.12.1-2.7M-PES').model
dist_model = CHGNet_Dist.from_existing(model).eval()
dist_model.enable_distributed_mode([5, 6])


In [23]:
pes = Potential_Dist(model=dist_model, num_threads=128)
pes(AseAtomsAdaptor().get_atoms(error_struct))

Bond graph is enabled but atom_cutoff is 6.000000 and bond_cutoff is 3.000000. The partition width is 6.512043 which is <= 2 * (atom_cutoff + bond_cutoff) (18.000000).
A wall width that is <= 2 * (atom_cutoff + bond_cutoff) will be inefficient but still yield correct results.
You should reduce the # of partitions. If you cannot fit your system on a reduced # of partitions, then your system is probably too dense.
Contact Kevin (kevinhan@cmu.edu) if you need help with this.
Distributed object creation: 0.11024336802074686


(tensor([[-12.7886]], device='cuda:5', grad_fn=<AddBackward0>),
 tensor([[ 0.7038, -0.2044,  0.0125],
         [-1.5125,  0.2765, -0.2434],
         [-1.1869, -1.1277,  1.5109],
         [ 0.0374, -0.4467,  0.2691],
         [ 0.2762, -0.6211, -0.1666],
         [ 0.0854, -0.3215,  0.1493],
         [ 0.1743, -0.1253, -0.7263],
         [-0.3499,  0.2539,  0.3457],
         [-0.2233, -1.5997, -0.2526],
         [-0.2424, -0.1144,  0.6065],
         [ 0.1811, -1.1773, -0.8434],
         [ 0.2107,  0.1117,  0.4116],
         [ 0.2106, -0.1045,  0.0894],
         [-0.7273,  0.4942,  0.4019],
         [-0.7177, -0.8574, -0.0876],
         [ 0.0459,  0.5048, -0.2253],
         [-0.4060, -1.8792, -0.7388],
         [ 1.1287,  2.2711, -0.4477],
         [ 0.0646,  0.1526,  0.3185],
         [-0.4481,  1.4451,  0.1728],
         [-0.2091,  0.0095, -0.0279],
         [-0.1161,  0.2185,  0.3378],
         [ 0.3741,  0.0751, -0.3116],
         [-0.1621, -0.1066, -0.0580],
         [ 0.6490, -0.01

In [ ]:
# Run on the prealloc
pes(LFP_atoms_prealloc)

In [ ]:
driver = MolecularDynamics(
    LFP_atoms,
    potential=pes,
    timestep=2,
    temperature=1000,
    # logfile="H2O_0.1M_1000k.log",
    # trajectory="H2O_0.1M_1000k.traj",
    loginterval=100000
)

driver.run(2000)

In [ ]:
import re
import matplotlib.pyplot as plt

# Read file content into a string
with open('nohup.out', 'r') as file:
    data = file.read()

# Regex patterns
allocated_pattern = re.compile(r"Allocated \(GB\): \[([^\]]+)\]")
reserved_pattern = re.compile(r"Reserved\s+\(GB\): \[([^\]]+)\]")
distributed_pattern = re.compile(r"Distributed object creation: ([\d.]+)")
step_time_pattern = re.compile(r"Step time: ([\d.]+)")

# Extract values
allocated_matches = allocated_pattern.findall(data)
reserved_matches = reserved_pattern.findall(data)
distributed_times = [float(x) for x in distributed_pattern.findall(data)]
step_times = [float(x) for x in step_time_pattern.findall(data)][1:]

# Compute averages
average_allocated = []
average_reserved = []

for alloc, resv in zip(allocated_matches, reserved_matches):
    alloc_vals = [float(val.strip().strip("'")) for val in alloc.split(',')]
    resv_vals = [float(val.strip().strip("'")) for val in resv.split(',')]
    average_allocated.append(sum(alloc_vals) / len(alloc_vals))
    average_reserved.append(sum(resv_vals) / len(resv_vals))

# Time steps
steps = list(range(len(average_allocated)))

# Plot 1: Memory usage
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(steps, average_allocated, label='Avg Allocated Memory', linewidth=2)
plt.plot(steps, average_reserved, label='Avg Reserved Memory', linewidth=2)
plt.title('Memory Usage per Step')
plt.xlabel('Step')
plt.ylabel('Memory (GB)')
plt.legend()
plt.grid(True)

# Plot 2: Timing info
plt.subplot(1, 2, 2)
plt.plot(steps, distributed_times, label='Distributed Object Creation Time', linewidth=2)
plt.plot(steps, step_times, label='Total Step Time', linestyle='--', linewidth=2)
plt.title('Timing Information per Step')
plt.xlabel('Step')
plt.ylabel('Time (s)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
